In [1]:
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os
import random
import datetime
import time

def mkdir(path):
    #判斷目錄是否存在
    #存在：True
    #不存在：False
    folder = os.path.exists(path)

    #判斷結果
    if not folder:
        #如果不存在，則建立新目錄
        os.makedirs(path)
        print(path + " 建立成功")

    else:
        print(path + " 目錄已存在")

        
def SeleniumGetPhoto(URL):
    CHROMEDRIVER_PATH = './tools/chromedriver'
    options = Options() # 啟動無頭模式
#     options.add_argument('--headless')  #規避google bug
#     options.add_argument('--ignore-certificate-errors')
#     options.add_argument('--disable-gpu')
#     options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(CHROMEDRIVER_PATH,options=options)
    driver.get("https://www.17ce.com/")
    time.sleep(3)
    driver.maximize_window()
    time.sleep(2)
    driver.find_element(By.ID, "nav1").click()
    driver.find_element(By.ID, "url").click()
    driver.find_element(By.ID, "url").send_keys(URL)
#     driver.find_element(By.ID, "su").click()
    driver.find_element_by_xpath('//*[@id="su"]').click()
    web = URL.replace("https://","").replace("/","")
    
    datename = time.strftime("%Y-%m-%d", time.localtime())
    mkdir("./data/"+web+"/photo/"+datename)
    
    time.sleep(45)  #要調回45秒
    driver.execute_script("document.body.style.zoom='0.8'") 
    time.sleep(5)  #要調回5秒
    driver.execute_script("window.scrollTo(0,230)")
    
    datefile = time.strftime(" %Y-%m-%d_%H_%M_%S", time.localtime())
    driver.save_screenshot( "./data/"+web+"/photo/"+datename+"/"+web+datefile+".png")
    soup = BeautifulSoup(driver.page_source,"html.parser")
    
    driver.close()
    print("完成：" + web)
    return web , soup

def RequestsGetcsv(web,soup):
    
    line = []
    fastestnode = []
    fastestsec = []
    slowestnode = []
    slowestsec = []
    average = []

    table = soup.find("tbody")

    for i in table.find_all("tr")[:6]:
        line.append(i.find_all("td")[0].text)
        fastestnode.append(i.find_all("td")[1].find("font",class_="fl").text)
        fastestsec.append(i.find_all("td")[1].find("font",class_="fr").text)
        slowestnode.append(i.find_all("td")[2].find("font",class_="fl").text)
        slowestsec.append(i.find_all("td")[2].find("font",class_="fr").text)
        average.append(i.find_all("td")[3].find("font",class_="fr").text)

    result_dict = {
        "URL":web,
        "時間":time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "線路":line,
        "最快節點":fastestnode,
        "最快節點秒數":fastestsec,
        "最慢節點":slowestnode,
        "最慢節點秒數":slowestsec,
        "平均響應":average

    }

    result = pd.DataFrame(result_dict,columns=["URL","時間","線路","最快節點","最快節點秒數","最慢節點","最慢節點秒數","平均響應"])
    
    filepath = "./data/"+web+"/"+web+".csv"
    if os.path.isfile(filepath):
        print("檔案存在。")
        result.to_csv("./data/"+web+"/"+web+".csv",encoding="utf_8_sig",mode='a',header=False,index=False)
    else:
        print("檔案不存在。")
        result.to_csv("./data/"+web+"/"+web+".csv",encoding="utf_8_sig",mode='a',index=False)
        
    print("完成紀錄 "+web+".csv")
    print("==========================")


In [2]:
web_list = pd.read_csv("./URL_dict.csv")
for i in web_list["URL"]:
    try:
        web,soup = SeleniumGetPhoto(i)
    except:
        print("SeleniumGetPhoto 有問題再跑一次")
        web,soup = SeleniumGetPhoto(i)
    
    try:
        RequestsGetcsv(web,soup)
    except:
        print("RequestsGetcsv 有問題再跑一次")
        RequestsGetcsv(web,soup)
    

./data/hqzb.tv/photo/2020-03-12 目錄已存在
完成：hqzb.tv
檔案存在。
完成紀錄 hqzb.tv.csv
./data/kl99.tv/photo/2020-03-12 目錄已存在
完成：kl99.tv
檔案存在。
完成紀錄 kl99.tv.csv
./data/ldsports.uk/photo/2020-03-12 目錄已存在
完成：ldsports.uk
檔案存在。
完成紀錄 ldsports.uk.csv
./data/ldzb8.tv/photo/2020-03-12 目錄已存在
完成：ldzb8.tv
檔案存在。
完成紀錄 ldzb8.tv.csv
./data/meiqiu.tv/photo/2020-03-12 目錄已存在
完成：meiqiu.tv
檔案存在。
完成紀錄 meiqiu.tv.csv
./data/pinqiu.tv/photo/2020-03-12 目錄已存在
完成：pinqiu.tv
檔案存在。
完成紀錄 pinqiu.tv.csv
./data/shanmao.tv/photo/2020-03-12 目錄已存在
完成：shanmao.tv
檔案存在。
完成紀錄 shanmao.tv.csv
./data/smtv.io/photo/2020-03-12 目錄已存在
完成：smtv.io
檔案存在。
完成紀錄 smtv.io.csv
./data/smzb.io/photo/2020-03-12 目錄已存在
完成：smzb.io
檔案存在。
完成紀錄 smzb.io.csv
